In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

# Hilfsfunktionen für die Spielimplementierung

In dieser Dateien werden Hilfsfunktionien deklariert, die für die grundlegende Spielimplementierung benötigt werden.

## Hilfsfunktionen für Spielsteine

Nachfolgend werden alle Hilfsfunktionen implementiert, die für das Interagieren mit den Spielsteinen benötigt werden.

Die Funktion `hasPlaceableStones` überprüft, ob ein Spieler für einen Zustand noch zusetzende Steine auf dem Stapel (_stash_) besitzt. Die Funktion hat zwei Argumente:

- `s` ist ein Zustand eines Spiels;
- `p` ist ein Spieler.

Die Funktion gibt ein booleschen Wert zurück.

In [ ]:
# Calculates wether or not a player still has stones which he has not placed yet
def hasPlaceableStones(s, p):
    # Extract the count of the placeable stones for black and white
    ((cw, cb), _) = s
    # Return wether or not the given player has at least one stone
    return cw >= 1 if p == 'w' else cb >= 1

Die Funktion `countStones` zählt die Steine eines Spieler auf einem Spielbrett. Die Funktion hat zwei Argumente:

- `s` ist ein Zustand eines Spiels;
- `p` ist ein Spieler.

Die Funktion zählt nur die Steine auf dem Brett, nicht die Steine auf dem Stapel und gibt diese als Ganzzahl zurück.

In [ ]:
# Counts how many stones the given player has left on the board
def countStones(s, p):
    # Extract the board from the state
    (_, board) = s
    # Count how many times player occurs on the board
    return [cell for ring in board for cell in ring].count(p)

Die Funktion `isAllowedToJump` überprüft, ob ein Spieler bei einem gegebenen Zustand seine Steine beliebig bewegen darf. Die Funktion hat zwei Argumente:

- `s` ist ein Zustand eines Spiels;
- `p` ist ein Spieler.

Ein Spieler darf genau dann seine Steine bewegen, wenn er weniger als drei Steine auf dem Spielbrett hat und sich keine Steine mehr von dem Spieler auf dem Stapel befinden. Die zweite Bedingung wird von der Funktion aber nicht überprüft, weil davon ausgegangen wird, dass die Funktion `hasPlaceableStones` zuvor aufgerufen wird.

Die Funktion gibt einen booleschen Wert zurück. 

In [ ]:
# Calculates wether or not the player is allowed to jump with his stones,
# instead of just moving them
def isAllowedToJump(s, p):
    return countStones(s, p) <= 3

Die Funktion `hasEnoughStones` überprüft, ob ein Spieler noch genügend Steine übrig hat. Die Funktion hat zwei Argumente:

- `s` ist ein Zustand eines Spiels;
- `p` ist ein Spieler.

Ein Spieler hat genau dann genügend Steine, wenn er noch Steine zum Setzen auf dem Stapel hat oder er mindestes drei Steine auf dem Spielbrett besitzt.

Die Funktion gibt einen booleschen Wert zurück.

In [ ]:
# Calculates if the player has stones left to place or enought (at least 3) stones to continue playing
def hasEnoughStones(s, p):
    return hasPlaceableStones(s, p) or countStones(s, p) >= 3

Die Funktion `removeFromStash` entfernt einen Stein von dem Stapel. Die Funktion hat zwei Argumente:

- `stash` ist ein Stapel;
- `p` ist ein Spieler, dessen Stein entfernt werden soll.

Die Funktion gibt den neuen Stapel zurück.

In [ ]:
# Removes a stone from the player's stash and returns the new stash
def removeFromStash(stash, p):
    return (stash[0] - (1 if p == 'w' else 0), stash[1] - (1 if p == 'b' else 0))

## Hilfsfunktionen für Spieler

In diesem Kapitel werden Hilfsfunktionen für die Spieler implementiert.

Die Funktion `opponent` nimmt einen Spieler und gibt den Gegenspieler zurück. Die Funktion hat ein Argument:

- `p` ist der aktuelle Spieler.

Da Mühle ein Zwei-Personen-Spiel ist, gibt es für die Funktion nur zwei Fälle:

- bei einem weißen Spieler `'w'` wird der Gegenspieler schwarz `'b'` zurückgegeben,
- ansonsten wird weiß `'w'` als Gegenspieler zurückgegeben.

In [ ]:
def opponent(p):
    return 'b' if p == 'w' else 'w'

Die Funktion `getPlayerAt` gibt den Spieler an der gegeben Koordinate des Spielbrettes zurück. Die Funktion hat zwei Argumente:

- `board` ist ein Spielbrett;
- `coord` ist eine Koordinate, die überprüft werden soll. 

Die Funktion gibt einen Spieler zurück. Falls an dieser Koordinate sich kein Spielerstein befinden sollte, wird entsprechend `' '` zurückgegeben.

In [ ]:
# Returns the player on the given coord
def getPlayerAt(board, coord):
    (r, c) = coord
    return board[r][c]

Die Funktion `playerPhase` berechnet für einen gegebenen Zustand und einen Spieler die aktuelle Phase des Spielers. Die Funktion hat zwei Argumente:

- `s` ist der aktuelle Zustand eines Spiels,
- `p` ist der aktuelle Spieler.

Die Funktion überprüft mit den beiden Hilfsfunktionen `hasPlaceableStones` und `isAllowedToJump` die Spielerphase und gibt diese als Ganzzahl zurück.

In [ ]:
# Returns the phase of the given player
#   1. The player has to place his stones
#   2. The player is only allowed to move the stones along the lines
#   3. The player is allowed to jump with his stones
def playerPhase(s, p):
    # If the player has still stones left to place, he is still in phase 1
    if hasPlaceableStones(s, p):
        return 1
    # If the player is allowed to jump with his stones, he is in the last phase, phase 3
    elif isAllowedToJump(s, p):
        return 3
    # Else he is in phase 2, where he can only move his stones
    else:
        return 2

## Cells

In [ ]:
# Returns a set of tuples containing all coordinates of the cells owned by p
# Set of Tuples(ring, cell)
def findCellsOf(board, p):
    # Iterate over all cells and select only empty cells
    return {(r, c) for r in range(0, 3) for c in range(0, 8) if board[r][c] == p}

In [ ]:
# Returns a set of tuples containing all coordinates of the empty cells
# Set of Tuples(ring, cell)
def findEmptyCells(board):
    # Empty cells are marked as ' '
    return findCellsOf(board, ' ')

In [ ]:
# Returns a set of tuples containing all coordinates of the empty cells neighboring the given cell
# Set of Tuples(ring, cell)
def findNeighboringEmptyCells(board, coordinates):
    (rootR, rootC) = coordinates
    return {(r, c) for (r, c) in findEmptyCells(board) if (r == rootR and (c == (rootC + 7) % 8 or c == (rootC + 1) % 8)) 
                                                        or (c % 2 == 1 and c == rootC and (r == rootR - 1 or r == rootR + 1)) }


In [ ]:
def place(board, coordinates, player):
    (r, c) = coordinates
    return tuple(
        tuple(
            player if (c == ic) and (r == ir) else board[ir][ic]
            for ic in range(0, 8)
        ) for ir in range(0, 3)
    )

In [ ]:
def move(board, src, des):
    src_r, src_c = src
    des_r, des_c = des
    content_src = board[src_r][src_c]
    content_des = board[des_r][des_c]
    return tuple(
        tuple(
            content_des if (r,c) == src else (content_src if (r,c) == des else board[r][c])
            for c in range(0, 8)
        ) for r in range(0, 3)
    )

## Mills

In [ ]:
# Returns the coordinates of all mills the given player has
# Set of Frozensets of Tuples(ring, cell)
def findMills(board, p):
    # Calculate all mills on the rings
    return {
        frozenset((r, (c+o)%8) for o in range(0, 3))
        # Iterate over all rings
        for r in range(0, 3)
        # Iterate over all corners
        for c in range(0, 8, 2)
        # All 3 following cells starting at the given corner have to belong to the player
        if all(
            cell == p
            # Iterate over all 3 cells of the given side (c) by appending the first element
            # as the last mill wrapps around
            for cell in (list(board[r]) + [board[r][0]])[c:c+3]
        )
    # Calculate all mills crossing the rings
    } | {
        frozenset((r, c) for r in range(0, 3))
        # Iterate over cells in the middle of a side
        for c in range(1, 8, 2)
        # All 3 cells in the middle of a given side have to belong to the player
        if all(
            board[r][c] == p
            # Iterate over all 3 rings
            for r in range(0, 3)
        )
    }

In [ ]:
# Returns the number of mills the player build in his turn.
# The board is the board after his turn, but bevor he executed his mills.
def countNewMills(board, oldMills, p):
    return len(findMills(board, p) - oldMills)

In [ ]:
# Returns all stones that are poundable
# A stone is poundable if it is not in an opponent's mill.
# If all the opponent's stones are in a mill, the function returns all the opponent's stones instead, because then all the stones are poundable
def getCellsPoundable(board, p):
    opponentCells = findCellsOf(board, opponent(p))
    
    opponentCellsInMills = {
        cell
        for mill in findMills(board, opponent(p))
        for cell in mill
    }
    
    return opponentCells - opponentCellsInMills if len(opponentCells - opponentCellsInMills) > 0 else opponentCells

In [ ]:
# Returns all possible boards after the player has pounded his allowed mills
# Pounding a mill means removing an opponent stone
def poundMills(board, count, p):
    if count <= 0:
        return { board }

    # Compute all boards with an opponent stone removed
    return {
        place(b, cell, ' ')
        for b in poundMills(board, count-1, p)
        for cell in findCellsOf(b, opponent(p))
        if cell in getCellsPoundable(b, p)
    }